Script to run implicit solvent MD simulations of proline

In [1]:
from simtk import unit, openmm
from simtk.openmm import app
import numpy as np
from sys import stdout
from openmmtools import integrators
import numpy as np

from sys import stdout
import mdtraj as md
#from openmmtools import integrators

In [2]:
#USE EITHER PDB FOR CIS OR TRANS SIMULATIONS
pdb = app.PDBFile('cis_pro.pdb')
#pdb = app.PDBFile('trans_pro_reindexed.pdb')

#USE THIS CODE TO CREATE A SYSTEM FROM SCRATCH 
# ff = app.ForceField('amber99sbildn.xml',"amber96_obc.xml")
# system = ff.createSystem(
#     topology=pdb.getTopology(), 
#     removeCMMotion=True,
#     nonbondedMethod=app.NoCutoff,
#     constraints=app.HBonds, 
#     rigidWater=True
#     )

with open('noconstraints_xmlsystem.txt') as f:
    xml = f.read()
system = openmm.XmlSerializer.deserialize(xml)


In [ ]:
## creates serialised openmm system file

# xml = openmm.XmlSerializer.serialize(system)
# xml_file = open('cis_pro_xmlsystem.txt','w')
# xml_file.write(xml)
# xml_file.close()

In [3]:
#defining parameters
temperature = 300.0 * unit.kelvin
collision_rate = 1.0 / unit.picosecond
timestep = 1.0 * unit.femtosecond
reportInterval = 2000
steps = 5E+8
fname = 'test'
parametersdict = {'Collision rate':collision_rate,'Temperature':temperature,'Timestep':timestep,'Report Interval':reportInterval}
import pickle
f_p = open(f'parameters{fname}.pkl','wb')
pickle.dump(parametersdict,f_p)
f_p.close

<function BufferedWriter.close>

In [ ]:
#Setting up the simulation
integrator = openmm.LangevinIntegrator(temperature,collision_rate,timestep)
properties_dict = {}
properties_dict["DeviceIndex"] = "2"
platform = openmm.Platform.getPlatform(2)

positions = pdb.getPositions(asNumpy=True).value_in_unit(unit.nanometer)

simulation = app.Simulation(pdb.getTopology(), system, integrator,platform,platformProperties=properties_dict)
simulation.context.setPositions(positions)
simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)

simulation.reporters.append(app.StateDataReporter(stdout, reportInterval, step=True, potentialEnergy=True,temperature=True))
#use this reporter to report kinetic and total energies
simulation.reporters.append(openmm.app.StateDataReporter(
    f'Trajectories/{fname}.txt', 
    reportInterval=reportInterval, 
    step=True, 
    potentialEnergy=True,
    kineticEnergy = True,
    totalEnergy = True,
    temperature=True,
))
simulation.reporters.append(app.DCDReporter(f'Trajectories/{fname}.dcd',reportInterval))

In [ ]:
#Running the simulation
simulation.step(steps)